In [1]:
import os
import numpy as np
import cv2

In [6]:
base_dir = "CUB_200_2011/images/"
data = []
labels = []

In [7]:
for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)

    if os.path.isdir(folder_path):
        # loop through each image in the folder
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)

            # read the image using OpenCV
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # assuming grayscale, change if you want color
            if img is not None:
                # resize to a standard size if needed (optional)
                img = cv2.resize(img, (64, 64))  # replace with desired size

                # append the image data and label to our lists
                data.append(img)
                labels.append(folder_name)

In [ ]:
data

[]

In [ ]:
data_np = np.array(data, dtype=np.float32) / 255.0  # normalize to [0, 1]
labels_np = np.array(labels)

In [ ]:
np.save("data.npy", data_np)
np.save("labels.npy", labels_np)